In [1]:
import os
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from tqdm import tqdm
from models_used import CIFAR10_Net
from torchvision import models
import json
from voting_utils import WeightFinding

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

Using device: cuda


In [3]:
# Define the directory containing the models
model_directory = 'cinic_various_models/'

# Function to load models with "fl" in their name
def load_models(directory, noise_type = 'fl'):
    models_loaded = []

    # List all files in the directory
    files = os.listdir(directory)
    
    for file in tqdm(files):
        if noise_type in file:
            # Load the model
            if 'cifar10_net' in file:
                model = CIFAR10_Net()
            else:
                model = models.vgg16(pretrained = True)
                input_lastLayer = model.classifier[6].in_features
                model.classifier[6] = nn.Linear(input_lastLayer,10)
            model_path = os.path.join(directory, file)
            model.load_state_dict(torch.load(model_path))
            models_loaded.append((file, model))
    
    return models_loaded

# Load the models
fl_models = load_models(model_directory)

# Example usage: Print the names of the loaded models
for file_name, model in fl_models:
    print(f"Loaded model from file: {file_name}")

  0%|          | 0/24 [00:00<?, ?it/s]/home/pratik/miniconda3/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/pratik/miniconda3/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
100%|██████████| 24/24 [00:06<00:00,  3.43it/s]

Loaded model from file: vgg_cifar_fl_1.pt
Loaded model from file: cifar10_net_imagenet_fl_2.pt
Loaded model from file: vgg_imagenet_fl_0.pt
Loaded model from file: vgg_cifar_fl_2.pt
Loaded model from file: cifar10_net_cifar_fl_1.pt
Loaded model from file: cifar10_net_imagenet_fl_0.pt
Loaded model from file: cifar10_net_cifar_fl_2.pt
Loaded model from file: cifar10_net_imagenet_fl_1.pt
Loaded model from file: vgg_cifar_fl_0.pt
Loaded model from file: vgg_imagenet_fl_2.pt
Loaded model from file: cifar10_net_cifar_fl_0.pt
Loaded model from file: vgg_imagenet_fl_1.pt


In [4]:
di_models = load_models(model_directory, noise_type = 'di')

# Example usage: Print the names of the loaded models
for file_name, model in di_models:
    print(f"Loaded model from file: {file_name}")
    # You can also print the model structure if needed
    # print(model)


100%|██████████| 24/24 [00:07<00:00,  3.32it/s]

Loaded model from file: cifar10_net_imagenet_di_0.pt
Loaded model from file: cifar10_net_cifar_di_0.pt
Loaded model from file: vgg_imagenet_di_1.pt
Loaded model from file: cifar10_net_cifar_di_2.pt
Loaded model from file: cifar10_net_imagenet_di_1.pt
Loaded model from file: vgg_imagenet_di_2.pt
Loaded model from file: vgg_cifar_di_2.pt
Loaded model from file: vgg_cifar_di_1.pt
Loaded model from file: vgg_imagenet_di_0.pt
Loaded model from file: vgg_cifar_di_0.pt
Loaded model from file: cifar10_net_cifar_di_1.pt
Loaded model from file: cifar10_net_imagenet_di_2.pt


### Load validation dataloaders

In [5]:
cinic_directory = 'cinic_10_data'
cinic_mean = [0.47889522, 0.47227842, 0.43047404]
cinic_std = [0.24205776, 0.23828046, 0.25874835]

cifar_valid = torch.utils.data.DataLoader(
    torchvision.datasets.ImageFolder(cinic_directory + '/valid2/cifar',
    	transform=transforms.Compose([transforms.ToTensor(),
        transforms.Normalize(mean=cinic_mean,std=cinic_std)])),
    batch_size=512, shuffle=False)

imagenet_valid = torch.utils.data.DataLoader(
    torchvision.datasets.ImageFolder(cinic_directory + '/valid2/imagenet',
    	transform=transforms.Compose([transforms.ToTensor(),
        transforms.Normalize(mean=cinic_mean,std=cinic_std)])),
    batch_size=512, shuffle=False)

### Create weight finder objects

In [6]:
fl_just_models = [model for file_name, model in fl_models]
di_just_models = [model for file_name, model in di_models]

fl_just_names = [file_name for file_name, model in fl_models]
di_just_names = [file_name for file_name, model in di_models]

In [7]:
ci_loaders = [cifar_valid, imagenet_valid]
c_loader = [cifar_valid]
i_loader = [imagenet_valid]

#### Uncomment the cell below only if you're computing the weight finder objects for the first time

In [8]:
wf_fl_ci = WeightFinding(fl_just_models, ci_loaders, device)
wf_di_ci = WeightFinding(di_just_models, ci_loaders, device)

wf_fl_c = WeightFinding(fl_just_models, c_loader, device)
wf_di_c = WeightFinding(di_just_models, c_loader, device)

wf_fl_i = WeightFinding(fl_just_models, i_loader, device)
wf_di_i = WeightFinding(di_just_models, i_loader, device)

# wf_fl_ci.models = []
# wf_di_ci.models = []
# wf_fl_c.models = []
# wf_di_c.models = []
# wf_fl_i.models = []
# wf_di_i.models = []

100%|██████████| 137/137 [00:15<00:00,  8.60it/s]


In [10]:
# Save these objects
wf_fl_ci.save('weight_finders/wf_fl_ci.pkl')
wf_di_ci.save('weight_finders/wf_di_ci.pkl')
# wf_fl_c.save('weight_finders/wf_fl_c.pkl')
# wf_di_c.save('weight_finders/wf_di_c.pkl')
# wf_fl_i.save('weight_finders/wf_fl_i.pkl')
# wf_di_i.save('weight_finders/wf_di_i.pkl')

# Load the objects
wf_fl_ci = WeightFinding.load('weight_finders/wf_fl_ci.pkl')
wf_di_ci = WeightFinding.load('weight_finders/wf_di_ci.pkl')
# wf_fl_c = WeightFinding.load('weight_finders/wf_fl_c.pkl')
# wf_di_c = WeightFinding.load('weight_finders/wf_di_c.pkl')
# wf_fl_i = WeightFinding.load('weight_finders/wf_fl_i.pkl')
# wf_di_i = WeightFinding.load('weight_finders/wf_di_i.pkl')

### LOO Weights

In [12]:
fl_loo_weights_borda_ci = wf_fl_ci.loo_pytorch(method = 'borda')
fl_loo_weights_borda_c = wf_fl_c.loo_pytorch(method = 'borda')
fl_loo_weights_borda_i = wf_fl_i.loo_pytorch(method = 'borda')

di_loo_weights_borda_ci = wf_di_ci.loo_pytorch(method = 'borda')
di_loo_weights_borda_c = wf_di_c.loo_pytorch(method = 'borda')
di_loo_weights_borda_i = wf_di_i.loo_pytorch(method = 'borda')

fl_loo_weights_plurality_ci = wf_fl_ci.loo_pytorch(method = 'plurality')
fl_loo_weights_plurality_c = wf_fl_c.loo_pytorch(method = 'plurality')
fl_loo_weights_plurality_i = wf_fl_i.loo_pytorch(method = 'plurality')

di_loo_weights_plurality_ci = wf_di_ci.loo_pytorch(method = 'plurality')
di_loo_weights_plurality_c = wf_di_c.loo_pytorch(method = 'plurality')
di_loo_weights_plurality_i = wf_di_i.loo_pytorch(method = 'plurality')

Calculating LOO values for 12 models
Calculating LOO values for 12 models
Calculating LOO values for 12 models
Calculating LOO values for 12 models
Calculating LOO values for 12 models
Calculating LOO values for 12 models
Calculating LOO values for 12 models
Calculating LOO values for 12 models
Calculating LOO values for 12 models
Calculating LOO values for 12 models
Calculating LOO values for 12 models
Calculating LOO values for 12 models


In [13]:
# make dictionary of the weights with model names
fl_loo_weights_borda_ci_dict = dict(zip(fl_just_names, fl_loo_weights_borda_ci))
fl_loo_weights_borda_c_dict = dict(zip(fl_just_names, fl_loo_weights_borda_c))
fl_loo_weights_borda_i_dict = dict(zip(fl_just_names, fl_loo_weights_borda_i))
di_loo_weights_borda_ci_dict = dict(zip(di_just_names, di_loo_weights_borda_ci))
di_loo_weights_borda_c_dict = dict(zip(di_just_names, di_loo_weights_borda_c))
di_loo_weights_borda_i_dict = dict(zip(di_just_names, di_loo_weights_borda_i))

fl_loo_weights_plurality_ci_dict = dict(zip(fl_just_names, fl_loo_weights_plurality_ci))
fl_loo_weights_plurality_c_dict = dict(zip(fl_just_names, fl_loo_weights_plurality_c))
fl_loo_weights_plurality_i_dict = dict(zip(fl_just_names, fl_loo_weights_plurality_i))
di_loo_weights_plurality_ci_dict = dict(zip(di_just_names, di_loo_weights_plurality_ci))
di_loo_weights_plurality_c_dict = dict(zip(di_just_names, di_loo_weights_plurality_c))
di_loo_weights_plurality_i_dict = dict(zip(di_just_names, di_loo_weights_plurality_i))

In [14]:
# Save the weights
with open('weights/loo/fl_loo_weights_borda_ci.json', 'w') as f:
    json.dump(fl_loo_weights_borda_ci_dict, f)

with open('weights/loo/fl_loo_weights_borda_c.json', 'w') as f:
    json.dump(fl_loo_weights_borda_c_dict, f)

with open('weights/loo/fl_loo_weights_borda_i.json', 'w') as f:
    json.dump(fl_loo_weights_borda_i_dict, f)

with open('weights/loo/di_loo_weights_borda_ci.json', 'w') as f:
    json.dump(di_loo_weights_borda_ci_dict, f)

with open('weights/loo/di_loo_weights_borda_c.json', 'w') as f:
    json.dump(di_loo_weights_borda_c_dict, f)

with open('weights/loo/di_loo_weights_borda_i.json', 'w') as f:
    json.dump(di_loo_weights_borda_i_dict, f)

with open('weights/loo/fl_loo_weights_plurality_ci.json', 'w') as f:
    json.dump(fl_loo_weights_plurality_ci_dict, f)

with open('weights/loo/fl_loo_weights_plurality_c.json', 'w') as f:
    json.dump(fl_loo_weights_plurality_c_dict, f)

with open('weights/loo/fl_loo_weights_plurality_i.json', 'w') as f:
    json.dump(fl_loo_weights_plurality_i_dict, f)

with open('weights/loo/di_loo_weights_plurality_ci.json', 'w') as f:
    json.dump(di_loo_weights_plurality_ci_dict, f)

with open('weights/loo/di_loo_weights_plurality_c.json', 'w') as f:
    json.dump(di_loo_weights_plurality_c_dict, f)

with open('weights/loo/di_loo_weights_plurality_i.json', 'w') as f:
    json.dump(di_loo_weights_plurality_i_dict, f)

# Load the weights
with open('weights/loo/fl_loo_weights_borda_ci.json', 'r') as f:
    fl_loo_weights_borda_ci_dict = json.load(f)

with open('weights/loo/fl_loo_weights_borda_c.json', 'r') as f:
    fl_loo_weights_borda_c_dict = json.load(f)

with open('weights/loo/fl_loo_weights_borda_i.json', 'r') as f:
    fl_loo_weights_borda_i_dict = json.load(f)

with open('weights/loo/di_loo_weights_borda_ci.json', 'r') as f:
    di_loo_weights_borda_ci_dict = json.load(f)

with open('weights/loo/di_loo_weights_borda_c.json', 'r') as f:
    di_loo_weights_borda_c_dict = json.load(f)

with open('weights/loo/di_loo_weights_borda_i.json', 'r') as f:
    di_loo_weights_borda_i_dict = json.load(f)

with open('weights/loo/fl_loo_weights_plurality_ci.json', 'r') as f:
    fl_loo_weights_plurality_ci_dict = json.load(f)

with open('weights/loo/fl_loo_weights_plurality_c.json', 'r') as f:
    fl_loo_weights_plurality_c_dict = json.load(f)

with open('weights/loo/fl_loo_weights_plurality_i.json', 'r') as f:
    fl_loo_weights_plurality_i_dict = json.load(f)

with open('weights/loo/di_loo_weights_plurality_ci.json', 'r') as f:
    di_loo_weights_plurality_ci_dict = json.load(f)

with open('weights/loo/di_loo_weights_plurality_c.json', 'r') as f:
    di_loo_weights_plurality_c_dict = json.load(f)

with open('weights/loo/di_loo_weights_plurality_i.json', 'r') as f:
    di_loo_weights_plurality_i_dict = json.load(f)

### CRH weights

In [15]:
fl_crh_weights_ci = wf_fl_ci.crh_pytorch()
fl_crh_weights_c = wf_fl_c.crh_pytorch()
fl_crh_weights_i = wf_fl_i.crh_pytorch()

di_crh_weights_ci = wf_di_ci.crh_pytorch()
di_crh_weights_c = wf_di_c.crh_pytorch()
di_crh_weights_i = wf_di_i.crh_pytorch()

In [16]:
fl_crh_weights_ci_dict = dict(zip(fl_just_names, fl_crh_weights_ci))
fl_crh_weights_c_dict = dict(zip(fl_just_names, fl_crh_weights_c))
fl_crh_weights_i_dict = dict(zip(fl_just_names, fl_crh_weights_i))
di_crh_weights_ci_dict = dict(zip(di_just_names, di_crh_weights_ci))
di_crh_weights_c_dict = dict(zip(di_just_names, di_crh_weights_c))
di_crh_weights_i_dict = dict(zip(di_just_names, di_crh_weights_i))

In [17]:
# Save the CRH weights
with open('weights/crh/fl_crh_weights_ci.json', 'w') as f:
    json.dump(fl_crh_weights_ci_dict, f)

with open('weights/crh/fl_crh_weights_c.json', 'w') as f:
    json.dump(fl_crh_weights_c_dict, f)

with open('weights/crh/fl_crh_weights_i.json', 'w') as f:
    json.dump(fl_crh_weights_i_dict, f)

with open('weights/crh/di_crh_weights_ci.json', 'w') as f:
    json.dump(di_crh_weights_ci_dict, f)

with open('weights/crh/di_crh_weights_c.json', 'w') as f:
    json.dump(di_crh_weights_c_dict, f)

with open('weights/crh/di_crh_weights_i.json', 'w') as f:
    json.dump(di_crh_weights_i_dict, f)

# Load the CRH weights
with open('weights/crh/fl_crh_weights_ci.json', 'r') as f:
    fl_crh_weights_ci_dict = json.load(f)

with open('weights/crh/fl_crh_weights_c.json', 'r') as f:
    fl_crh_weights_c_dict = json.load(f)

with open('weights/crh/fl_crh_weights_i.json', 'r') as f:
    fl_crh_weights_i_dict = json.load(f)

with open('weights/crh/di_crh_weights_ci.json', 'r') as f:
    di_crh_weights_ci_dict = json.load(f)

with open('weights/crh/di_crh_weights_c.json', 'r') as f:
    di_crh_weights_c_dict = json.load(f)

with open('weights/crh/di_crh_weights_i.json', 'r') as f:
    di_crh_weights_i_dict = json.load(f)

### Entropy weights

In [18]:
fl_entropy_weights_ci = wf_fl_ci.entropy_weights()
fl_entropy_weights_c = wf_fl_c.entropy_weights()
fl_entropy_weights_i = wf_fl_i.entropy_weights()

di_entropy_weights_ci = wf_di_ci.entropy_weights()
di_entropy_weights_c = wf_di_c.entropy_weights()
di_entropy_weights_i = wf_di_i.entropy_weights()

In [19]:
fl_entropy_weights_ci_dict = dict(zip(fl_just_names, fl_entropy_weights_ci))
fl_entropy_weights_c_dict = dict(zip(fl_just_names, fl_entropy_weights_c))
fl_entropy_weights_i_dict = dict(zip(fl_just_names, fl_entropy_weights_i))
di_entropy_weights_ci_dict = dict(zip(di_just_names, di_entropy_weights_ci))
di_entropy_weights_c_dict = dict(zip(di_just_names, di_entropy_weights_c))
di_entropy_weights_i_dict = dict(zip(di_just_names, di_entropy_weights_i))

In [20]:
# Save the Entropy weights
with open('weights/entropy/fl_entropy_weights_ci.json', 'w') as f:
    json.dump(fl_entropy_weights_ci_dict, f)

with open('weights/entropy/fl_entropy_weights_c.json', 'w') as f:
    json.dump(fl_entropy_weights_c_dict, f)

with open('weights/entropy/fl_entropy_weights_i.json', 'w') as f:
    json.dump(fl_entropy_weights_i_dict, f)

with open('weights/entropy/di_entropy_weights_ci.json', 'w') as f:
    json.dump(di_entropy_weights_ci_dict, f)

with open('weights/entropy/di_entropy_weights_c.json', 'w') as f:
    json.dump(di_entropy_weights_c_dict, f)

with open('weights/entropy/di_entropy_weights_i.json', 'w') as f:
    json.dump(di_entropy_weights_i_dict, f)

# Load the entropy weights
with open('weights/entropy/fl_entropy_weights_ci.json', 'r') as f:
    fl_entropy_weights_ci_dict = json.load(f)

with open('weights/entropy/fl_entropy_weights_c.json', 'r') as f:
    fl_entropy_weights_c_dict = json.load(f)

with open('weights/entropy/fl_entropy_weights_i.json', 'r') as f:
    fl_entropy_weights_i_dict = json.load(f)

with open('weights/entropy/di_entropy_weights_ci.json', 'r') as f:
    di_entropy_weights_ci_dict = json.load(f)

with open('weights/entropy/di_entropy_weights_c.json', 'r') as f:
    di_entropy_weights_c_dict = json.load(f)

with open('weights/entropy/di_entropy_weights_i.json', 'r') as f:
    di_entropy_weights_i_dict = json.load(f)

### Accuracy weights

In [21]:
fl_accuracy_weights_ci = wf_fl_ci.accuracy_weights()
fl_accuracy_weights_c = wf_fl_c.accuracy_weights()
fl_accuracy_weights_i = wf_fl_i.accuracy_weights()

di_accuracy_weights_ci = wf_di_ci.accuracy_weights()
di_accuracy_weights_c = wf_di_c.accuracy_weights()
di_accuracy_weights_i = wf_di_i.accuracy_weights()

In [22]:
fl_accuracy_weights_ci_dict = dict(zip(fl_just_names, fl_accuracy_weights_ci))
fl_accuracy_weights_c_dict = dict(zip(fl_just_names, fl_accuracy_weights_c))
fl_accuracy_weights_i_dict = dict(zip(fl_just_names, fl_accuracy_weights_i))
di_accuracy_weights_ci_dict = dict(zip(di_just_names, di_accuracy_weights_ci))
di_accuracy_weights_c_dict = dict(zip(di_just_names, di_accuracy_weights_c))
di_accuracy_weights_i_dict = dict(zip(di_just_names, di_accuracy_weights_i))

In [23]:
# Save the accuracy weights
with open('weights/accuracy/fl_accuracy_weights_ci.json', 'w') as f:
    json.dump(fl_accuracy_weights_ci_dict, f)

with open('weights/accuracy/fl_accuracy_weights_c.json', 'w') as f:
    json.dump(fl_accuracy_weights_c_dict, f)

with open('weights/accuracy/fl_accuracy_weights_i.json', 'w') as f:
    json.dump(fl_accuracy_weights_i_dict, f)

with open('weights/accuracy/di_accuracy_weights_ci.json', 'w') as f:
    json.dump(di_accuracy_weights_ci_dict, f)

with open('weights/accuracy/di_accuracy_weights_c.json', 'w') as f:
    json.dump(di_accuracy_weights_c_dict, f)

with open('weights/accuracy/di_accuracy_weights_i.json', 'w') as f:
    json.dump(di_accuracy_weights_i_dict, f)

# Load the accuracy weights
with open('weights/accuracy/fl_accuracy_weights_ci.json', 'r') as f:
    fl_accuracy_weights_ci_dict = json.load(f)

with open('weights/accuracy/fl_accuracy_weights_c.json', 'r') as f:
    fl_accuracy_weights_c_dict = json.load(f)

with open('weights/accuracy/fl_accuracy_weights_i.json', 'r') as f:
    fl_accuracy_weights_i_dict = json.load(f)

with open('weights/accuracy/di_accuracy_weights_ci.json', 'r') as f:
    di_accuracy_weights_ci_dict = json.load(f)

with open('weights/accuracy/di_accuracy_weights_c.json', 'r') as f:
    di_accuracy_weights_c_dict = json.load(f)

with open('weights/accuracy/di_accuracy_weights_i.json', 'r') as f:
    di_accuracy_weights_i_dict = json.load(f)

### Shapley weights

In [26]:
fl_shapley_weights_borda_ci = wf_fl_ci.shapley_pytorch(method = 'borda')
fl_shapley_weights_borda_c = wf_fl_c.shapley_pytorch(method = 'borda')
fl_shapley_weights_borda_i = wf_fl_i.shapley_pytorch(method = 'borda')

di_shapley_weights_borda_ci = wf_di_ci.shapley_pytorch(method = 'borda')
di_shapley_weights_borda_c = wf_di_c.shapley_pytorch(method = 'borda')
di_shapley_weights_borda_i = wf_di_i.shapley_pytorch(method = 'borda')

fl_shapley_weights_plurality_ci = wf_fl_ci.shapley_pytorch(method = 'plurality')
fl_shapley_weights_plurality_c = wf_fl_c.shapley_pytorch(method = 'plurality')
fl_shapley_weights_plurality_i = wf_fl_i.shapley_pytorch(method = 'plurality')

di_shapley_weights_plurality_ci = wf_di_ci.shapley_pytorch(method = 'plurality')
di_shapley_weights_plurality_c = wf_di_c.shapley_pytorch(method = 'plurality')
di_shapley_weights_plurality_i = wf_di_i.shapley_pytorch(method = 'plurality')

Calculating Shapley values for 12 models


  0%|          | 0/13 [00:00<?, ?it/s]/home/pratik/miniconda3/lib/python3.12/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/pratik/miniconda3/lib/python3.12/site-packages/numpy/core/_methods.py:121: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(
100%|██████████| 12/12 [00:00<00:00, 723.35it/s]


Calculating Shapley values for 12 models


  0%|          | 0/13 [00:00<?, ?it/s]/home/pratik/miniconda3/lib/python3.12/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/pratik/miniconda3/lib/python3.12/site-packages/numpy/core/_methods.py:121: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(
100%|██████████| 12/12 [00:00<00:00, 711.52it/s]


Calculating Shapley values for 12 models


...

100%|██████████| 12/12 [00:00<00:00, 710.61it/s]


Calculating Shapley values for 12 models


100%|██████████| 12/12 [00:00<00:00, 695.74it/s]


Calculating Shapley values for 12 models


100%|██████████| 12/12 [00:00<00:00, 707.37it/s]


In [27]:
fl_shapley_weights_borda_ci_dict = dict(zip(fl_just_names, fl_shapley_weights_borda_ci))
fl_shapley_weights_borda_c_dict = dict(zip(fl_just_names, fl_shapley_weights_borda_c))
fl_shapley_weights_borda_i_dict = dict(zip(fl_just_names, fl_shapley_weights_borda_i))
di_shapley_weights_borda_ci_dict = dict(zip(di_just_names, di_shapley_weights_borda_ci))
di_shapley_weights_borda_c_dict = dict(zip(di_just_names, di_shapley_weights_borda_c))
di_shapley_weights_borda_i_dict = dict(zip(di_just_names, di_shapley_weights_borda_i))

fl_shapley_weights_plurality_ci_dict = dict(zip(fl_just_names, fl_shapley_weights_plurality_ci))
fl_shapley_weights_plurality_c_dict = dict(zip(fl_just_names, fl_shapley_weights_plurality_c))
fl_shapley_weights_plurality_i_dict = dict(zip(fl_just_names, fl_shapley_weights_plurality_i))
di_shapley_weights_plurality_ci_dict = dict(zip(di_just_names, di_shapley_weights_plurality_ci))
di_shapley_weights_plurality_c_dict = dict(zip(di_just_names, di_shapley_weights_plurality_c))
di_shapley_weights_plurality_i_dict = dict(zip(di_just_names, di_shapley_weights_plurality_i))

In [28]:
# Save the weights
with open('weights/shapley/fl_shapley_weights_borda_ci.json', 'w') as f:
    json.dump(fl_shapley_weights_borda_ci_dict, f)

with open('weights/shapley/fl_shapley_weights_borda_c.json', 'w') as f:
    json.dump(fl_shapley_weights_borda_c_dict, f)

with open('weights/shapley/fl_shapley_weights_borda_i.json', 'w') as f:
    json.dump(fl_shapley_weights_borda_i_dict, f)

with open('weights/shapley/di_shapley_weights_borda_ci.json', 'w') as f:
    json.dump(di_shapley_weights_borda_ci_dict, f)

with open('weights/shapley/di_shapley_weights_borda_c.json', 'w') as f:
    json.dump(di_shapley_weights_borda_c_dict, f)

with open('weights/shapley/di_shapley_weights_borda_i.json', 'w') as f:
    json.dump(di_shapley_weights_borda_i_dict, f)

with open('weights/shapley/fl_shapley_weights_plurality_ci.json', 'w') as f:
    json.dump(fl_shapley_weights_plurality_ci_dict, f)

with open('weights/shapley/fl_shapley_weights_plurality_c.json', 'w') as f:
    json.dump(fl_shapley_weights_plurality_c_dict, f)

with open('weights/shapley/fl_shapley_weights_plurality_i.json', 'w') as f:
    json.dump(fl_shapley_weights_plurality_i_dict, f)

with open('weights/shapley/di_shapley_weights_plurality_ci.json', 'w') as f:
    json.dump(di_shapley_weights_plurality_ci_dict, f)

with open('weights/shapley/di_shapley_weights_plurality_c.json', 'w') as f:
    json.dump(di_shapley_weights_plurality_c_dict, f)

with open('weights/shapley/di_shapley_weights_plurality_i.json', 'w') as f:
    json.dump(di_shapley_weights_plurality_i_dict, f)

# Load the weights
# with open('weights/shapley/fl_shapley_weights_borda_ci.json', 'r') as f:
#     fl_shapley_weights_borda_ci_dict = json.load(f)

# with open('weights/shapley/fl_shapley_weights_borda_c.json', 'r') as f:
#     fl_shapley_weights_borda_c_dict = json.load(f)

# with open('weights/shapley/fl_shapley_weights_borda_i.json', 'r') as f:
#     fl_shapley_weights_borda_i_dict = json.load(f)

# with open('weights/shapley/di_shapley_weights_borda_ci.json', 'r') as f:
#     di_shapley_weights_borda_ci_dict = json.load(f)

# with open('weights/shapley/di_shapley_weights_borda_c.json', 'r') as f:
#     di_shapley_weights_borda_c_dict = json.load(f)

# with open('weights/shapley/di_shapley_weights_borda_i.json', 'r') as f:
#     di_shapley_weights_borda_i_dict = json.load(f)

# with open('weights/shapley/fl_shapley_weights_plurality_ci.json', 'r') as f:
#     fl_shapley_weights_plurality_ci_dict = json.load(f)

# with open('weights/shapley/fl_shapley_weights_plurality_c.json', 'r') as f:
#     fl_shapley_weights_plurality_c_dict = json.load(f)

# with open('weights/shapley/fl_shapley_weights_plurality_i.json', 'r') as f:
#     fl_shapley_weights_plurality_i_dict = json.load(f)

# with open('weights/shapley/di_shapley_weights_plurality_ci.json', 'r') as f:
#     di_shapley_weights_plurality_ci_dict = json.load(f)

# with open('weights/shapley/di_shapley_weights_plurality_c.json', 'r') as f:
#     di_shapley_weights_plurality_c_dict = json.load(f)

# with open('weights/shapley/di_shapley_weights_plurality_i.json', 'r') as f:
#     di_shapley_weights_plurality_i_dict = json.load(f)

### Regression weights

In [11]:
fl_regression_weights_ci = wf_fl_ci.regression_pytorch()
# fl_regression_weights_c = wf_fl_c.regression_pytorch()
# fl_regression_weights_i = wf_fl_i.regression_pytorch()

di_regression_weights_ci = wf_di_ci.regression_pytorch()
# di_regression_weights_c = wf_di_c.regression_pytorch()
# di_regression_weights_i = wf_di_i.regression_pytorch()

Calculating Regression weights for 12 models
Calculating Regression weights for 12 models


In [12]:
fl_regression_weights_ci_dict = dict(zip(fl_just_names, fl_regression_weights_ci))
# fl_regression_weights_c_dict = dict(zip(fl_just_names, fl_regression_weights_c))
# fl_regression_weights_i_dict = dict(zip(fl_just_names, fl_regression_weights_i))
di_regression_weights_ci_dict = dict(zip(di_just_names, di_regression_weights_ci))
# di_regression_weights_c_dict = dict(zip(di_just_names, di_regression_weights_c))
# di_regression_weights_i_dict = dict(zip(di_just_names, di_regression_weights_i))

In [13]:
# Save the weights
with open('weights/regression/fl_regression_weights_ci.json', 'w') as f:
    json.dump(fl_regression_weights_ci_dict, f)

# with open('weights/regression/fl_regression_weights_c.json', 'w') as f:
#     json.dump(fl_regression_weights_c_dict, f)

# with open('weights/regression/fl_regression_weights_i.json', 'w') as f:
#     json.dump(fl_regression_weights_i_dict, f)

with open('weights/regression/di_regression_weights_ci.json', 'w') as f:
    json.dump(di_regression_weights_ci_dict, f)

# with open('weights/regression/di_regression_weights_c.json', 'w') as f:
#     json.dump(di_regression_weights_c_dict, f)

# with open('weights/regression/di_regression_weights_i.json', 'w') as f:
#     json.dump(di_regression_weights_i_dict, f)

# Load the weights
# with open('weights/regression/fl_regression_weights_ci.json', 'r') as f:
#     fl_regression_weights_ci_dict = json.load(f)

# with open('weights/regression/fl_regression_weights_c.json', 'r') as f:
#     fl_regression_weights_c_dict = json.load(f)

# with open('weights/regression/fl_regression_weights_i.json', 'r') as f:
#     fl_regression_weights_i_dict = json.load(f)

# with open('weights/regression/di_regression_weights_ci.json', 'r') as f:
#     di_regression_weights_ci_dict = json.load(f)

# with open('weights/regression/di_regression_weights_c.json', 'r') as f:
#     di_regression_weights_c_dict = json.load(f)

# with open('weights/regression/di_regression_weights_i.json', 'r') as f:
#     di_regression_weights_i_dict = json.load(f)